In [ ]:
import os, sys
from tqdm import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import zipfile
import time
import shutil
import collections
from pathlib import Path

In [ ]:
import warnings
warnings.filterwarnings('ignore')

### Loading packages

In [ ]:
import sys
from pathlib import Path

here_path = Path().resolve()
repo_path = here_path.parents[1]
sys.path.append(str(repo_path))

In [ ]:
from py.utils import verifyDir, verifyFile, verifyType

In [ ]:
import graphviz
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from py.config import Config

cfg = Config()

np.random.seed(cfg.RANDOM_STATE)
cfg.DATA_PATH, cfg.MODEL_PATH

In [ ]:
QSCORE_PATH=f"{cfg.DATA_PATH}pp1/Qscores/"
IMAGES_PATH = f"{cfg.DATA_PATH}pp1/images/"
MODEL_PATH = f"{cfg.MODEL_PATH}pp1/{cfg.YEAR_STUDIED}/cnn/"
EXPLAIN_PATH = f"{cfg.MODEL_PATH}pp1/{cfg.YEAR_STUDIED}/explanations/"

In [ ]:
verifyDir(MODEL_PATH)
verifyDir(EXPLAIN_PATH)

### Verify GPU

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch_type = torch.float32 if device.type == "cuda" else torch.float16
device, torch_type

### Loading data

In [ ]:
NUM_CLASSES = 1 if "reg" in cfg.ML_TASK else 2

In [ ]:
%%time
data_df = pd.read_csv(f"{QSCORE_PATH}scores.csv", sep=";", low_memory=False)
data_df["image_path"] = f"{IMAGES_PATH}{cfg.YEAR_STUDIED}/" + data_df["image_path"]
data_df["image_id"] = data_df["image_id"].apply(str)
data_df.sort_values(by=[cfg.PERCEPTION_METRIC], ascending=False, inplace=True)
data_df

### Loading Model

In [ ]:
from py.models.classification.cnn.vgg import VGG16

model = VGG16(num_classes=2, use_mlp=True)
model.load_state_dict(torch.load(f"{MODEL_PATH}{cfg.MODEL_FEATURE_NAME}_best_model.pth"))
model.eval()

#### Image test

In [ ]:
from py.models.datasets.transformations import ImageTransforms

transforms_list = ImageTransforms().get(model_name=cfg.MODEL_FEATURE_NAME)
transforms_list

In [ ]:
from PIL import Image

original_image = Image.open(data_df["image_path"].tolist()[0]).convert("RGB")
original_image

In [ ]:
target_class = 1 # not safety

### GradCAM (Gradient-weighted Class Activation Mapping)

- **What it does**: Shows which parts of the image the model is "looking at" when making predictions
- **How it works**: Uses gradients flowing into the last convolutional layer to highlight important regions
- **Speed**: Very fast (~0.1 seconds per image)
- **Best for**: Understanding spatial attention and localization

In [ ]:
from py.models.explainers import GradCAM

gradcam = GradCAM(model)

In [ ]:
# Generate explanation (input_tensor: 1x3x224x224, original_image: HxWx3)
cam_heatmap, gradcam_predicted_class = gradcam.generate_cam(
                                            original_image,
                                            transforms_list=transforms_list["val"],
                                            target_class=target_class  # None = use predicted class, or specify a class index
                                        )

#### GradCAM Interpretation

- **Red/Yellow regions**: Areas the model focuses on most
- **Blue regions**: Areas the model ignores
- **Heatmap**: Shows spatial importance across the image
- **Good for**: "Where is the model looking?"

In [ ]:
# Generate explanation (input_tensor: 1x3x224x224, original_image: HxWx3)
gradcam_viz, cam_resized = gradcam.visualize(
                    original_image,
                    cam_heatmap,
                )

In [ ]:
plt.imshow(gradcam_viz)
plt.title(f'Predicted: Class {gradcam_predicted_class}')
plt.show()

### LIME (Local Interpretable Model-agnostic Explanations)

- **What it does**: Highlights which image regions contribute most to the prediction
- **How it works**: Perturbs the image and sees how predictions change
- **Speed**: Slower (~30-60 seconds per image with 1000 samples)
- **Best for**: Understanding feature importance in a human-interpretable way

In [ ]:
from py.models.explainers import ImageLIME

# Create LIME explainer
lime_explainer = ImageLIME(model)

In [ ]:
%%time
explanation, lime_predicted_class = lime_explainer.explain(
                                        original_image,
                                        transforms_list["val"],
                                        target_class=target_class,
                                        num_samples=1000,  # More samples = better but slower
                                    )

#### LIME Interpretation

- **Green regions**: Features that increase the class probability
- **Red regions**: Features that decrease the class probability
- **Superpixels**: Image is divided into interpretable segments
- **Good for**: "Which features matter for this prediction?"

In [ ]:
lime_viz = lime_explainer.visualize(original_image, 
                                   explanation,
                                   target_class=lime_predicted_class,
                                   num_features=10,      # Show top 10 superpixels
                                   positive_only=False   # Show both positive and negative contributions
                                )

In [ ]:
plt.imshow(lime_viz)
plt.title(f'LIME: Class {lime_predicted_class}')
plt.show()

#### Visualize both

In [ ]:
# Create figure
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Original image
axes[0].imshow(original_image)
axes[0].set_title('Original Image')
axes[0].axis('off')

# GradCAM
axes[1].imshow(gradcam_viz)
axes[1].set_title(f'GradCAM: class {gradcam_predicted_class}')
axes[1].axis('off')

# LIME
axes[2].imshow(lime_viz)
axes[2].set_title(f'LIME: class {lime_predicted_class}')
axes[2].axis('off')

plt.tight_layout()
plt.show()